In [29]:
import pandas as pd
import numpy as np
import igraph as ig
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from ipywidgets import interactive
import seaborn as sns
import os
import ast

In [16]:
#check if running directly from main file or indirectly from a file outside this directory
# Get the current working directory
current_directory = os.getcwd()
if current_directory.endswith('601project'):
    data = pd.read_csv("Pokemon.csv")
else:
    data = pd.read_csv("../Pokemon.csv")

# Define custom colors for each type
type_colors = {
    'Bug': 'lightgreen',
    'Dark': 'black',
    'Dragon': 'darkblue',
    'Electric': 'yellow',
    'Fairy': 'lightpink',
    'Fighting': 'red',
    'Fire': 'orange',
    'Flying': 'lightblue',
    'Ghost': 'purple',
    'Grass': 'green',
    'Ground': 'brown',
    'Ice': 'white',
    'Normal': 'grey',
    'Poison': 'mediumvioletred',
    'Psychic': 'lightcoral',
    'Rock': 'beige',
    'Steel': 'silver',
    'Water': 'blue',
}

In [44]:
if current_directory.endswith('601project'):
    pokemon_data = pd.read_csv("pokemon-data-mega.csv")
else:
    pokemon_data = pd.read_csv("../pokemon-data-mega.csv")
#pokemon_data

# Select only the "Name" and "Next Evolution(s)" columns
pokemon_evolutions = pokemon_data[['Name', 'Next Evolution(s)']]

# Convert the string representation of the list to an actual list
pokemon_evolutions.loc[:, 'Next Evolution(s)'] = pokemon_evolutions['Next Evolution(s)'].apply(ast.literal_eval)

# Unstack the "Next Evolution(s)" column
unstacked_pokemon_evolutions = pokemon_evolutions.explode('Next Evolution(s)')

# Display the resulting DataFrame
print(unstacked_pokemon_evolutions)


                 Name Next Evolution(s)
0           Abomasnow    Abomasnow-Mega
1      Abomasnow-Mega               NaN
2                Abra           Kadabra
3               Absol        Absol-Mega
4          Absol-Mega               NaN
..                ...               ...
914          Zweilous         Hydreigon
915           Zygarde       Zygarde-10%
915           Zygarde  Zygarde-Complete
916       Zygarde-10%               NaN
917  Zygarde-Complete               NaN

[956 rows x 2 columns]


In [3]:
# Adding a new column 'evolution' with values 1-4 repeating
data['Evolution'] = [1, 2, 3, 4] * (len(data) // 4)
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Evolution
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,1
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,2
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,3
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,4
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,1


In [26]:

# Define the effectiveness data
#Attacking Type is by columns, defending type by rows
type_effectiveness_data = {
    'Attacking_Type': ['Normal', 'Fire', 'Water', 'Electric', 'Grass', 'Ice', 'Fighting', 'Poison', 'Ground',
                       'Flying', 'Psychic', 'Bug', 'Rock', 'Ghost', 'Dragon', 'Dark', 'Steel', 'Fairy'],
    'Normal': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.0, 1.0, 1.0, 0.5, 1.0],
    'Fire': [1.0, 0.5, 0.5, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5, 1.0, 2.0, 1.0],
    'Water': [1.0, 2.0, 0.5, 1.0, 0.5, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0, 1.0, 1.0],
    'Electric': [1.0, 1.0, 2.0, 0.5, 0.5, 1.0, 1.0, 1.0, 0.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0],
    'Grass': [1.0, 0.5, 2.0, 0.5, 0.5, 1.0, 1.0, 0.5, 2.0, 0.5, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 0.5, 1.0],
    'Ice': [1.0, 0.5, 0.5, 1.0, 2.0, 0.5, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0],
    'Fighting': [2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0, 0.5, 0.5, 0.5, 2.0, 0.0, 1.0, 2.0, 2.0, 0.5],
    'Poison': [1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 0.5, 0.5, 1.0, 2.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.0, 2.0],
    'Ground': [1.0, 2.0, 1.0, 2.0, 0.5, 1.0, 1.0, 2.0, 1.0, 0.0, 1.0, 0.5, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0],
    'Flying': [1.0, 1.0, 1.0, 0.5, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.5, 1.0, 1.0, 1.0, 0.5, 1.0],
    'Psychic': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 0.0, 0.5, 1.0],
    'Bug': [1.0, 0.5, 1.0, 1.0, 2.0, 1.0, 0.5, 0.5, 1.0, 0.5, 2.0, 1.0, 1.0, 0.5, 1.0, 2.0, 0.5, 0.5],
    'Rock': [1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0],
    'Ghost': [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0, 1.0],
    'Dragon': [1.0, 0.5, 0.5, 0.5, 0.5, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 0.0],
    'Dark': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0, 0.5],
    'Steel': [1.0, 0.5, 0.5, 0.5, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.5, 1.0, 0.5, 2.0],
    'Fairy': [1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 0.5, 2.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 0.0, 2.0, 0.5, 1.0]
}
#lengths = [len(normal), len(fire), len(water), len(elec), len(grass), len(ice), len(fighting), len(poison), len(ground), len(flying), len(psychic), len(bug), len(rock), len(ghost), len(dragon), len(dark), len(steel), len(fairy)]

# Create the DataFrame
pokemon_effectiveness_df = pd.DataFrame(type_effectiveness_data).set_index('Attacking_Type').transpose()

# Display the DataFrame
#print(pokemon_effectiveness_df)
pokemon_effectiveness_df


Attacking_Type,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
Normal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.5,1.0
Fire,1.0,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,2.0,1.0
Water,1.0,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0,1.0
Electric,1.0,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0
Grass,1.0,0.5,2.0,0.5,0.5,1.0,1.0,0.5,2.0,0.5,1.0,0.5,2.0,1.0,0.5,1.0,0.5,1.0
Ice,1.0,0.5,0.5,1.0,2.0,0.5,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0
Fighting,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,0.5,0.5,0.5,2.0,0.0,1.0,2.0,2.0,0.5
Poison,1.0,1.0,1.0,1.0,0.5,1.0,1.0,0.5,0.5,1.0,2.0,1.0,0.5,0.5,1.0,1.0,0.0,2.0
Ground,1.0,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,0.0,1.0,0.5,2.0,1.0,1.0,1.0,2.0,1.0
Flying,1.0,1.0,1.0,0.5,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,0.5,1.0


In [24]:
def Combat_Power(attack,special_atk,defense,special_def,hp):
    total_a = attack + special_atk
    total_d = defense + special_def
    numerator = math.floor(total_a * (total_d**0.5)*(hp**0.5)*(0.667934**2)) # Taking the average level as 25 so used the cpm value for level 25
    return math.floor(max(10,numerator/10))

#https://pokemongohub.net/post/wiki/cp-mechanics/
#https://pokemongohub.net/post/article/pokemon-go-cpm-list/

In [7]:
data['CP'] = data.apply(lambda row: Combat_Power(row['Attack'], row['Sp. Atk'], row['Defense'], row['Sp. Def'], row['HP']), axis=1)

data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Evolution,CP
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,1,364
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,2,586
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,3,982
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,4,1380
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,1,300
